In [123]:
import pandas as pd
import numpy as np
import zipfile
import pymysql
from sqlalchemy import create_engine

In [124]:
df = pd.read_csv('/Users/anielkaaslan/Desktop/GSAF5.csv', encoding='cp1252', sep=',')
columns = df.columns
columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [125]:
# Examine the data for potential issues. DUPLICATES
select_columns = ['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23']

df = df[select_columns].drop_duplicates()#  ==== 1
df.head()


# df_ages = df.DataFrame('ages_15to25', 'ages_26to35', 'ages_36to45')

# print(df.describe())

# df_by_age = df.groupby('Age')
# print(df_by_age)
# 
# drop columns case2 and case 3, pdf, href formula and href
# for 

# change type to 4 values, 1 2 3 or 4 depending on case
# df.columns
# drop_cols = list(null_cols[null_cols > 5000].index)
# df = df.drop(drop_cols, axis=1)
# # def whereNotNull(df, col):
# #     return df[col].loc[df[col].notnull()]
# # df['age'] = whereNotNull(df, 'age').apply(lambda x: age[x])
# # where name in name = male or female, fill with NOT RECORDED
# if x in name == male or female:

# df['missing_number?'] = np.where(df['Nums'].isnull(), 'Yes', 'No')
# df['age', 'time', 'species'] = df[['age', 'time', 'species']].fillna()

# change column order

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN


In [126]:
null_cols = df.isnull().sum()#  ==== 2 
null_cols[null_cols > 0]
drop_cols = list(null_cols[null_cols > 3000].index)
df = df.drop(drop_cols, axis=1)
df.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Injury,Fatal (Y/N),Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,Minor injury to thigh,N,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,Lacerations to hands,N,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,Lacerations to lower leg,N,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,Struck by fin on chest & leg,N,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,No injury: Knocked off board by shark,N,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989


In [127]:
df = (df.drop(columns = ['pdf', 'Name', 'href formula', 'href', 'Case Number.1', 'Case Number.2', 'original order']))#  ==== 3
print(df.columns)

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Species ',
       'Investigator or Source'],
      dtype='object')


In [128]:
df['Age'] = df['Age'].str.extract("([+]?\d*\.\d+|[+]?\d+)").astype(float) #extract values in Age that are not numeric

df['Age'] = df['Age'].fillna(value=0)#  ==== 4 add value 0 to all empty cells in Age

df.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Sex,Age,Injury,Fatal (Y/N),Species,Investigator or Source
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,16.0,Minor injury to thigh,N,NaN,"Orlando Sentinel, 9/19/2016"
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,36.0,Lacerations to hands,N,NaN,"Orlando Sentinel, 9/19/2016"
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,43.0,Lacerations to lower leg,N,NaN,"Orlando Sentinel, 9/19/2016"
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,M,0.0,Struck by fin on chest & leg,N,NaN,"The Age, 9/18/2016"
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,M,0.0,No injury: Knocked off board by shark,N,2 m shark,"The Age, 9/16/2016"


In [129]:
df['Age'] = df['Age'].str.extract("([+]?\d*\.\d+|[+]?\d+)").astype(float) #extract values in Age that are not numeric
df['Age'] = df['Age'].fillna(value=0)#  ==== 4
print((df['Age']))
print(df.dtypes)
df['Age'] = df['Age'].astype('int') # change data type in Age


AttributeError: Can only use .str accessor with string values, which use np.object_ dtype in pandas

In [130]:
print(df['Age'].min())
print(df['Age'].max())

relation = df.groupby('Age')['Fatal (Y/N)'].count()
relation.head()

0.0
87.0


Age
0.0    2700
1.0       1
2.0       2
3.0       4
5.0       7
Name: Fatal (Y/N), dtype: int64

In [131]:
ages_15to25 = df.query('15 <= Age <= 26')
ages_26to35 = df.query('26 <= Age <= 35')
ages_36to45 = df.query('36 <= Age <= 45')
ages_46to55 = df.query('46 <= Age <= 55')
ages_56to65 = df.query('56 <= Age <= 65')
ages_66andup = df.query('66 <= Age')
                       
ages_66andup.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Sex,Age,Injury,Fatal (Y/N),Species,Investigator or Source
21,2016.07.28.R,28-Jul-16,2016,Unprovoked,CHINA,Hong Kong,NaN,Swimming,M,72.0,Lacerations to leg,N,NaN,"Klassick, 7/28/2016"
72,2016.04.09,09-Apr-16,2016,Unprovoked,NEW CALEDONIA,Grand Terre,Poe Beach,Walking,F,69.0,FATAL,Y,"Tiger shark, 2.5 m",Les Nouvelles Caledonnie. 4/11/2016
105,2015.12.22,22-Dec-15,2015,Unprovoked,USA,Hawaii,La'aloa Beach Park,Paddle boarding,M,71.0,"No injury, shark bit board",N,9' shark,"West Hawaii Today, 12/23/2015"
176,2015.07.01,01-Jul-15,2015,Unprovoked,USA,North Carolina,"Ocracoke, Lifeguard Beach, National Park Servi...",Swimming,M,68.0,"Injuries to torso, hip, lower leg & hands",N,6' to 7' shark,"C. Creswell, GSAF"
211,2015.04.24.c,24-Jun-15,2015,Unprovoked,AUSTRALIA,New South Wales,"Belongil Beach, Byron Bay",Surf skiing,M,71.0,"No injury, knocked off ski",N,"White shark, 3 m","Echo Daily, 6/25/2015"
